In [1]:
import requests
import pandas as pd
import plotly.graph_objs as go
from typing import Annotated, List
from datetime import datetime, date

In [2]:
def nyfed_recession_prob():
    url = "https://www.newyorkfed.org/medialibrary/media/research/capital_markets/allmonth.xls"
    full_file_path = r"C:\Users\chris\trade\curr_pos\treasuries\nyfed_recession_prob.xlsx"
    try:    
        response = requests.get(url)
        if response.status_code == 200:
            with open(full_file_path, "wb") as f:
                chunk = response.content
                f.write(chunk)
            return pd.read_excel(full_file_path, parse_dates=["Date"])
        else:
            raise Exception(f"Bad Status: {response.status}")
    except Exception as e:
        print(e)
        return pd.DataFrame()

In [3]:
def plot_term_spread(df: pd.DataFrame, date_subset_range: Annotated[List[date], 2] | None = None):
    df_copy = df.copy()
    df_copy["Date"] = pd.to_datetime(df_copy["Date"])
    if date_subset_range:
        df_copy = df_copy[
            (df_copy["Date"] >= date_subset_range[0])
            & (df_copy["Date"] <= date_subset_range[1])
        ]
        
    df_copy["term_spread"] = df_copy["10 Year Treasury Yield"] - df_copy["3 Month Treasury Yield"]
    df_copy["term_spread_beb"] = df_copy["10 Year Treasury Yield"] - df_copy["3 Month Treasury Yield (Bond Equivalent Basis)"]
    fig = go.Figure()
    
    dates = df_copy["Date"].tolist()
    term_spread = df_copy["term_spread"].tolist()
    term_spread_beb = df_copy["term_spread_beb"].tolist()
    
    fig.add_trace(
        go.Scatter(x=dates, y=term_spread, mode="lines+markers", name=f"Spread")
    )
    fig.add_trace(
        go.Scatter(x=dates, y=term_spread_beb, mode="lines+markers", name="BEB")
    )

    fig.update_layout(
        title="Treasury Spread: 10 yr bond rate-3 month bill rate",
        xaxis_title="Date",
        yaxis_title="Spread (basis points)",
        showlegend=False,
    )

    fig.show()
    return df_copy


def plot_recession_prob(df: pd.DataFrame, date_subset_range: Annotated[List[date], 2] | None = None):
    df_copy = df.copy()
    df_copy["Date"] = pd.to_datetime(df_copy["Date"])
    if date_subset_range:
        df_copy = df_copy[
            (df_copy["Date"] >= date_subset_range[0])
            & (df_copy["Date"] <= date_subset_range[1])
        ]
        
    dates = df_copy["Date"].tolist()
    term_spread = df_copy["Rec_prob"].tolist()
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(x=dates, y=term_spread, mode="lines+markers", name=f"Treasury Spread")
    )

    fig.update_layout(
        title="Probability of U.S. Recession, Twelve Months Ahead of Term Spread Readings",
        xaxis_title="Date",
        yaxis_title="Percent (monthly average)",
        showlegend=False,
    )

    fig.show()
    return df_copy

In [5]:
df = nyfed_recession_prob()
df.head()

,Date,10 Year Treasury Yield,3 Month Treasury Yield,3 Month Treasury Yield (Bond Equivalent Basis),Spread,Rec_prob,NBER_Rec
0,1959-01-31,4.02,2.82,2.879694,1.140306,NaN,0.0
1,1959-02-28,3.96,2.70,2.756312,1.203688,NaN,0.0
2,1959-03-31,3.99,2.80,2.859125,1.130875,NaN,0.0
3,1959-04-30,4.12,2.95,3.013443,1.106557,NaN,0.0
4,1959-05-31,4.31,2.84,2.900265,1.409735,NaN,0.0


In [6]:
date_range = [datetime(2020, 1, 1), datetime(2023, 12, 1)]

In [7]:
df_term_spread = plot_term_spread(df, date_range)

In [8]:
df_recession_prob = plot_recession_prob(df)